In [1]:
using DelimitedFiles

In [2]:
mutable struct Guard
    ID
    M_Sleep    
end

In [3]:
function fill_in_guards(dict)
    current_guard = 0
    for entry in dict
        if entry[2][1] != 0
            current_guard = entry[2][1]
        else
            entry[2][1] = current_guard
        end
    end
    return dict
end

fill_in_guards (generic function with 1 method)

In [4]:
function get_entry_dict(m)
    entry_dict = Dict()
    for i = 1:size(m,1)
        year, month, day = split(m[i,1], r"-|\[")[2:4]
        hour, minute = split(m[i,2], r":|\]")[1:2]
        action = m[i,3]
        if action == "Guard"
            guard = split(m[i,4], r"#")[2]
            action = "begins"
        else
            guard = "0"
        end
        
        timestamp = [year, month, day, hour*minute]
        info = [parse(Int, guard), action]
        entry_dict[timestamp] = info
        
    end
    sorted_dict = sort(collect(entry_dict),  by=x->x[1])
    final_dict = fill_in_guards(sorted_dict)
    return final_dict
end

get_entry_dict (generic function with 1 method)

In [5]:
function get_guards(entries)
    guards = []
    for i = 1:length(entries)
        info = entries[i][2]
        time = entries[i][1]
        if info[2] == "wakes"
            minutes = []
            sleep_start = parse(Int, entries[i-1][1][4])
            sleep_end = parse(Int, time[4])
            for i = sleep_start:(sleep_end - 1)
                push!(minutes, i)
            end
            flag = false
            for guard in guards
                if guard.ID == info[1]
                    flag = true
                    guard.M_Sleep = vcat(guard.M_Sleep, minutes)
                    break
                end
            end
            if !flag
                push!(guards, Guard(info[1], minutes))
            end
        end
    end
    return guards
end

get_guards (generic function with 1 method)

In [6]:
function puzzle_1(guards)
    longest = 0
    target = 0
    for guard in guards
        if length(guard.M_Sleep) > longest
            longest = length(guard.M_Sleep)
            target = guard
        end
    end
    print("Target ID: ", target.ID)
    target_minute = 0
    most_hits = 0
    unique_minutes = unique(collect(target.M_Sleep))
    for minute in unique_minutes
        hits = count(x->x==minute, target.M_Sleep)
        if hits > most_hits
            most_hits = hits
            target_minute = minute
        end
    end
    print("\nTarget Minute: ",target_minute)
    answer = target.ID * target_minute
    print("\nAnswer: ", answer)
end

puzzle_1 (generic function with 1 method)

In [7]:
function puzzle_2(guards)
    most_hits = 0
    target_minute = 0
    target = 0
    for guard in guards
        unique_minutes = unique(collect(guard.M_Sleep))
        for minute in unique_minutes
            hits = count(x->x==minute, guard.M_Sleep)
            if hits > most_hits
                most_hits = hits
                target_minute = minute
                target = guard.ID
            end
        end
    end
    print("\nTarget ID: ", target)
    print("\nTarget Minute: ",target_minute)
    answer = target * target_minute
    print("\nAnswer: ", answer)
end

puzzle_2 (generic function with 1 method)

In [8]:
m = readdlm("input.txt");
entries = get_entry_dict(m);
guards = get_guards(entries)
print("PUZZLE #1:\n")
puzzle_1(guards)
print("\nPUZZLE #2:")
puzzle_2(guards)

PUZZLE #1:
Target ID: 2917
Target Minute: 25
Answer: 72925
PUZZLE #2:
Target ID: 1489
Target Minute: 33
Answer: 49137